In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
def parser(x):
    return datetime.strptime(x , '%d/%m/%Y')
ColumnList=[]

series= read_csv('DP26-skus monthly sales (FILTERED2).csv', parse_dates=['Date'] , index_col='Date', date_parser=parser)
for columns in series:
    ColumnList.append(columns)
print (ColumnList)
print(series.head())
series.plot()
pyplot.show()

In [ ]:
model={}
ColName=[]
movingavg_mape=[]
simpleExpo_mape=[]
DoubleExpo_mape=[]
M_tripleExpo_mape=[]
A_tripleExpo_mape=[]
movingavg_conf=[]
simpleExpo_conf=[]
DoubleExpo_conf=[]
M_tripleExpo_conf=[]
A_tripleExpo_conf=[]
movingavg_sum_error=[]
simpleExpo_sum_error=[]
DoubleExpo_sum_error=[]
M_tripleExpo_sum_error=[]
A_tripleExpo_sum_error=[]
movingavg_forecast_sum=[]
simpleExpo_forecast_sum=[]
DoubleExpo_forecast_sum=[]
M_tripleExpo_forecast_sum=[]
A_tripleExpo_forecast_sum=[]
model['movingavg_mape']=movingavg_mape
model['simpleExpo_mape']=simpleExpo_mape
model['DoubleExpo_mape']=DoubleExpo_mape
model['M_tripleExpo_mape']=M_tripleExpo_mape
model['A_tripleExpo_mape']=A_tripleExpo_mape
model['movingavg_conf']=movingavg_conf
model['simpleExpo_conf']=simpleExpo_conf
model['DoubleExpo_conf']=DoubleExpo_conf
model['M_tripleExpo_conf']=M_tripleExpo_conf
model['A_tripleExpo_conf']=A_tripleExpo_conf
model['movingavg_absolute_error']=movingavg_sum_error
model['simpleExpo_absolute_error']=simpleExpo_sum_error
model['DoubleExpo_absolute_error']=DoubleExpo_sum_error
model['M_tripleExpo_absolute_error']=M_tripleExpo_sum_error
model['A_tripleExpo_absolute_error']=A_tripleExpo_sum_error
model['ColName']=ColName
model['movingavg forecast sum']=movingavg_forecast_sum
model['simpleExpo forecast sum']=simpleExpo_forecast_sum
model['DoubleExpo forecast sum']=DoubleExpo_forecast_sum
model['M_tripleExpo forecast sum']=M_tripleExpo_forecast_sum
model['A_tripleExpo forecast sum']=A_tripleExpo_forecast_sum

In [ ]:
model2={}
movingavg_forecast=[]
simpleExpo_forecast=[]
DoubleExpo_forecast=[]
M_tripleExpo_forecast=[]
A_tripleExpo_forecast=[]
SKU=[]
model2['movingavg_forecast']=movingavg_forecast
model2['simpleExpo_forecast']=simpleExpo_forecast
model2['doubleExpo_forecast']=DoubleExpo_forecast
model2['M_tripleExpo_forecast']=M_tripleExpo_forecast
model2['A_tripleExpo_forecast']=A_tripleExpo_forecast
model2['SKU']=SKU

In [ ]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (((np.sum(np.abs(y_true - y_pred))) / (np.sum(y_pred))) * 100)

In [ ]:
def movingavg(series,l,u):
    min = 999999999999999
    for w in range (l,u+1):
        result=[]
        for j in range(w):
            result.append(series[j])
        for i in range (w,len(series)):
            result.append((sum(series[i-w:i]))/w)
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minW=w
            minr=result
        print ('Window-size {} - mape {}'.format(w,mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('Min mape is {} - window size {} - absolute error {} - forecast sum {}'.format(mape,minW,absolute_error,forecast_sum))
    movingavg_mape.append(min)
    movingavg_conf.append(minW)
    movingavg_sum_error.append(absolute_error)
    movingavg_forecast_sum.append(forecast_sum)
    movingavg_forecast.extend(minr)
    return minr
def exponential_smoothing(series, alphaL,alphaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        result = [series[0]] # first value is same as series
        for n in range(1, (len(series))):
            result.append((alpha/10.0) * series[n-1] + (1 - (alpha/10.0)) * result[n-1])
        mape=mean_absolute_percentage_error(series[24:],result[24:])
        #print result
        if (0<=mape<min):
            min=mape
            minalpha=(alpha/10.0)
            minr=result
        print ('alpha {} - mape {}'.format((alpha/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('best alpha {} - mape is {} - absolute error {} - forecast sum {}'.format(minalpha,min,absolute_error,forecast_sum))
    simpleExpo_mape.append(min)
    simpleExpo_conf.append(minalpha)
    simpleExpo_sum_error.append(absolute_error)
    simpleExpo_forecast_sum.append(forecast_sum)
    simpleExpo_forecast.extend(minr)
    return minr

def double_exponential_smoothing(series, alphaL, alphaU, betaL, betaU):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    min = 999999999999999
    
    for alpha in range (alphal, alphau+1):
        result=[]
        for beta in range (betal, betau+1):
            result = [series[0], series[0]]
            for n in range(2, len(series)):
                if n == 2:
                    level, trend = series[0], series[1] - series[0]
                if n <= len(series): # we are forecasting
                    value = series[n-1]
                last_level, level = level, (alpha/10.0)*value + (1-(alpha/10.0))*(level+trend)
                trend = (beta/10.0)*(level-last_level) + (1-(beta/10.0))*trend
                result.append(level+trend)
            mape=mean_absolute_percentage_error(series[24:],result[24:])
            #print result
            if (0<=mape<min):
                min=mape
                minalpha=(alpha/10.0)
                minbeta=(beta/10.0)
                minr=result
            print ('alpha {} - beta {} - mape {}'.format((alpha/10.0),(beta/10.0),mape))
    absolute_error=(min/100)*sum(minr[24:])
    forecast_sum=sum(minr[24:])
    print ('min alpha {} - min beta {} - mape {} - absolute error {} - forecast sum {}'.format(minalpha,minbeta,min,absolute_error,forecast_sum))
    DoubleExpo_mape.append(min)
    DoubleExpo_conf.append([minalpha,minbeta])
    DoubleExpo_sum_error.append(absolute_error)
    DoubleExpo_forecast_sum.append(forecast_sum)
    DoubleExpo_forecast.extend(minr)
    return minr

def M_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def M_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def M_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = M_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = M_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) * seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val/seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val/smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append((smooth+trend)*seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    M_tripleExpo_mape.append(min2)
    M_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    M_tripleExpo_sum_error.append(absolute_error)
    M_tripleExpo_forecast_sum.append(forecast_sum)
    M_tripleExpo_forecast.extend(bestr)
    return bestr

def A_initial_trend(series, slen):
    sum = 0.0
    for i in range(slen):
        sum += float(series[i+slen] - series[i]) / slen
    return sum / slen
def A_initial_seasonal_components(series, slen):
    seasonals = {}
    season_averages = []
    n_seasons = int(len(series)/slen)
    # compute season averages
    for j in range(n_seasons):
        season_averages.append(sum(series[slen*j:slen*j+slen])/float(slen))
    # compute initial values
    for i in range(slen):
        sum_of_vals_over_avg = 0.0
        for j in range(n_seasons):
            sum_of_vals_over_avg += series[slen*j+i]-season_averages[j]
        seasonals[i] = sum_of_vals_over_avg/n_seasons
    return seasonals
def A_triple_exponential_smoothing(series, slenL, slenU, alphaL, alphaU, betaL, betaU, gammaL, gammaU, n_preds):
    alphal=int(10*alphaL)
    alphau=int(10*alphaU)
    betal=int(10*betaL)
    betau=int(10*betaU)
    gammal=int(10*gammaL)
    gammau=int(10*gammaU)
    min = 999999999999999
    min2=999999999999999
    for slen in range (slenL, slenU+1):
        try:
            for alpha in range (alphal, alphau+1):
                for beta in range (betal, betau+1):
                    for gamma in range (gammal, gammau+1):
                        result=[]
                        seasonals = A_initial_seasonal_components(series, slen)
                        for i in range(len(series)+n_preds):
                            if i == 0: # initial values
                                smooth = series[0]
                                trend = A_initial_trend(series, slen)
                                result.append(series[0])
                                continue
                            if i >= len(series): # we are forecasting
                                m = i - len(series) + 1
                                result.append((smooth + m*trend) + seasonals[i%slen])
                            else:
                                val = series[i-1]
                                last_smooth, smooth = smooth, (alpha/10.0)*(val-seasonals[i%slen]) + (1-(alpha/10.0))*(smooth+trend)
                                trend = (beta/10.0) * (smooth-last_smooth) + (1-(beta/10.0))*trend
                                seasonals[i%slen] = (gamma/10.0)*(val-smooth) + (1-(gamma/10.0))*seasonals[i%slen]
                                result.append(smooth+trend+seasonals[i%slen])
                        mape=mean_absolute_percentage_error(series[24:],result[24:])
                        #print result
                        if (0<=mape<min):
                            min=mape
                            minalpha=(alpha/10.0)
                            minbeta=(beta/10.0)
                            mingamma=(gamma/10.0)
                            minr=result
                        print ('alpha {} - beta {} - gamma {} - mape {}'.format((alpha/10.0),(beta/10.0),(gamma/10.0),mape))
            print('best alpha {} - best beta {} - best gamma {} - mape {} - slen {}'.format(minalpha,minbeta,mingamma,min,slen))
        except:
            continue
        if (0<=min<min2):
            min2=min
            bestalpha=minalpha
            bestbeta=minbeta
            bestgamma=mingamma
            bestslen=slen
            bestr=minr
    absolute_error=(min2/100)*sum(bestr[24:])
    forecast_sum=sum(bestr[24:])
    print ('A {} - B {} - G {} - M {} - S {} - AE {} - FS {}'.format(bestalpha,bestbeta,bestgamma,min2,bestslen,absolute_error,forecast_sum))
    A_tripleExpo_mape.append(min2)
    A_tripleExpo_conf.append([bestalpha,bestbeta,bestgamma])
    A_tripleExpo_sum_error.append(absolute_error)
    A_tripleExpo_forecast_sum.append(forecast_sum)
    A_tripleExpo_forecast.extend(bestr)
    return bestr

In [ ]:
def forecasting ():
    for i in range(len(ColumnList)):
        ColName.append(ColumnList[i])
        for ik in range(0,36):
            SKU.append(ColumnList[i])
        print (ColumnList[i])
        print (i)
        print ("")
        movingavg(series[ColumnList[i]],2,12)
        exponential_smoothing(series[ColumnList[i]],0.1,0.9)
        double_exponential_smoothing(series[ColumnList[i]],0.1,0.9,0.1,0.9)
        M_triple_exponential_smoothing(series[ColumnList[i]],2,14,0.1,0.9,0.1,0.9,0.1,0.9,0)
        A_triple_exponential_smoothing(series[ColumnList[i]],2,14,0.1,0.9,0.1,0.9,0.1,0.9,0)

In [ ]:
forecasting()

In [ ]:
print (model2)
df=pd.DataFrame(model2)
print (df)
df.to_csv('python skus output with forecast values.csv', sep=',')

In [ ]:
print (model2)
df1=pd.DataFrame(model2)
print (df1)